In [1]:
try:
    import google.colab
    # We're in Colab
    !pip install git+https://github.com/landoskape/syd.git

except ImportError:
    pass

In [2]:
%matplotlib widget

# These are the imports we need for the viewer in this example
import random
import numpy as np
import matplotlib.pyplot as plt
from syd import make_viewer

In [3]:
# Start by using the make_viewer function to create a viewer. 
# It won't do much if you deploy it now, but this is how to start. 
viewer = make_viewer()

# Add parameters to the viewer. 
# The way this works is that you use the add_{parameter_type} function to add a parameter to the viewer. 
# Each parameter requires a name (e.g. "frequency", "sine_amplitude", etc).
# Each parameter (except for the button) has a value that you need to set when adding it. 
# Most parameters have additional required arguments like min_value, max_value, options, etc.
viewer.add_float("frequency", value=1.0, min_value=0.1, max_value=5.0)
viewer.add_float("sine_amplitude", value=1.0, min_value=0.1, max_value=2.0)
viewer.add_float("square_amplitude", value=1.0, min_value=0.1, max_value=2.0)
viewer.add_float("sawtooth_amplitude", value=1.0, min_value=0.1, max_value=2.0)
viewer.add_selection("sine_color", value="red", options=["red", "blue", "green", "black"])
viewer.add_selection("square_color", value="blue", options=["red", "blue", "green", "black"])
viewer.add_selection("sawtooth_color", value="green", options=["red", "blue", "green", "black"])
viewer.add_multiple_selection("waveform_type", value=["sine", "square", "sawtooth"], options=["sine", "square", "sawtooth"])
viewer.add_boolean("show_legend", value=True)
viewer.add_boolean("show_grid", value=True)

In [4]:
# Now, we need a plotting function. The plot function is called whenever parameters change
# and is what is used to actually make the figure you want to display. 

# Plot functions ~have~ to take two arguments: viewer and state. 
# (Advanced note: when using a subclass, you can use def plot(self, state) instead!)

# The viewer is just the viewer object you created earlier. You usually don't need this for the plot function.
# The state is a dictionary of all the current values of the parameters. 
# Use these to make your plot. 

# Notes on the plot function:
# - You should create a new figure each time, don't reuse old ones
# - Access parameter values using state['param_name']
# - Access your viewer class using "viewer" (or self if it's a bound method of a subclass)
# - Return the figure object, don't call plt.show()!

# Here's a plot function based on the viewer we made above:
def plot(viewer, state):
    """Plot the waveform based on current parameters."""
    t = np.linspace(0, 2 * np.pi, 1000)

    ymin = float("inf")
    ymax = float("-inf")

    fig, ax = plt.subplots()
    if "sine" in state["waveform_type"]:
        ax.plot(
            t,
            state["sine_amplitude"] * np.sin(state["frequency"] * t),
            color=state["sine_color"],
            label="Sine",
        )
        ymin = min(ymin, -state["sine_amplitude"])
        ymax = max(ymax, state["sine_amplitude"])
    if "square" in state["waveform_type"]:
        ax.plot(
            t,
            state["square_amplitude"] * np.sign(np.sin(state["frequency"] * t)),
            color=state["square_color"],
            label="Square",
        )
        ymin = min(ymin, -state["square_amplitude"])
        ymax = max(ymax, state["square_amplitude"])
    if "sawtooth" in state["waveform_type"]:
        ax.plot(
            t,
            state["sawtooth_amplitude"]
            * (t % (2 * np.pi / state["frequency"]))
            * (state["frequency"] / 2 / np.pi),
            color=state["sawtooth_color"],
            label="Sawtooth",
        )
        ymin = min(ymin, -state["sawtooth_amplitude"])
        ymax = max(ymax, state["sawtooth_amplitude"])

    ax.set_xlabel("Time")
    ax.set_ylabel("Amplitude")
    ax.grid(state["show_grid"])
    ax.set_ylim(ymin * 1.1, ymax * 1.1)
    if state["show_legend"]:
        ax.legend()
    return fig


# Now, we need to set the plot function. 
# This is how you do it!
viewer.set_plot(plot)

In [5]:
# Let's also make some callbacks so the viewer is a bit more interesting
# These are the two functions that we'll use as callbacks. 
# They are just like plot in that they take two arguments: viewer and state. 
# (Advanced note: same rule as before, if it's a subclass bound method, use this kind of signature: def callback(self, state))

# Callback methods can do anything, but most of the time they're used to update viewer's parameters. 
# To update parameters, use the viewer.update_{parameter_type} methods. These are essentially the same
# as the add_{parameter_type} methods, they have exactly the same input arguments. 
# The first input argument is the name of the parameter you want to update. 
# The other arguments are included whenever you want to update them. In these two examples, 
# we're updating the value of the parameter -- but you can also update the min_value, max_value, options, etc.

# In this one - we figure out whether the legend is being shown, then update the grid parameter accordingly.
def update_grid(viewer, state):
    showing_details = state["show_legend"]
    viewer.update_boolean("show_grid", value=showing_details)

# In this one - we figure out whether the grid is being shown, then update the legend parameter accordingly.
def update_legend(viewer, state):
    showing_details = state["show_grid"]
    viewer.update_boolean("show_legend", value=showing_details)

# Now, we need to add the callbacks to the viewer. 
# This is how you do it!
# The on_change method takes two arguments: the name of the parameter you want to listen to, and the callback function.
# This means that any time the parameter changes, the callback function will be called.
# For this first one, if you interact with the show_legend parameter, the update_grid function will be called.
viewer.on_change("show_legend", update_grid)
# For this second one, if you interact with the show_grid parameter, the update_legend function will be called.
viewer.on_change("show_grid", update_legend)


# Let's also make one where the color of each waveform is converted to black if it exceeds a certain amplitude.
amplitude_limit = 1.5
def mark_high_amplitude(viewer, state):
    # Here, we check if the amplitude of each waveform exceeds the limit.
    # If it does, we update the color of that specific waveform to black.
    if state["sine_amplitude"] > amplitude_limit:
        viewer.update_selection("sine_color", value="black")
    if state["square_amplitude"] > amplitude_limit:
        viewer.update_selection("square_color", value="black")
    if state["sawtooth_amplitude"] > amplitude_limit:
        viewer.update_selection("sawtooth_color", value="black")

# We want the color to be updated whenever the amplitude changes of ~any~ of the waveforms, so we can be fancy:
viewer.on_change(["sine_amplitude", "square_amplitude", "sawtooth_amplitude"], mark_high_amplitude)

# Voila! On change accepts a list of parameter names, and will activate the callback whenever any of them change. 

In [6]:
# Finally, we can also add a button to perform some action. 
# You know the drill by now, we need to define the function which accepts two arguments: viewer and state. 
# The function will be called whenever the button is clicked. 
# In this case, we're updating the colors of the waveforms.
# In reality, you can do anything you want in the callback - it's super useful to do things like:
# - Save a copy of the figure in your current state:
#    (to do this, use fig = viewer.plot(state) to get the figure, then save it using fig.savefig("filename.png"))
# - Print the current state to whatever console you're using. For example, suppose you're looking through possible
#   parameter values and want to save the ones you like. You can print to screen, or save the current state to a file.
def randomize_colors(viewer, state):
    colors = ["red", "blue", "green"]
    random.shuffle(colors)
    viewer.update_selection("sine_color", value=colors[0])
    viewer.update_selection("square_color", value=colors[1])
    viewer.update_selection("sawtooth_color", value=colors[2])

# Add button to randomize colors
# The add_button method takes three arguments: the name of the button, the label of the button, and the callback function.
# The name is what's used to identify the button in the viewer. 
# The label is what's displayed on the button in the GUI interface.
# The callback is the function that will be called when the button is clicked.
# You can use update_button just like for the other parameters. 
viewer.add_button("randomize_colors", label="Randomize Colors", callback=randomize_colors)

In [ ]:
# Now we deploy! 
# You can just run viewer.deploy() to use the default settings, however, I'm going to use this extra one:
# Continuous update: this means that the plot will be updated continuously as you interact with the viewer.
# If your plot method is a bit slower, it's a better idea to set continuous=False (which is default).
viewer.deploy(continuous=False)